In [1]:

%load_ext autoreload
%autoreload 2


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

In [3]:
train_dataset, val_dataset, train_loader, val_loader = CIFAR10()

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"



# train standard model 

In [6]:
from torch.utils.tensorboard import SummaryWriter
net_ori = BaseNet().to(device)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_ori.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base.pth'
device = "cuda"
writer = SummaryWriter("NormalBase")

In [ ]:
train(net_ori, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 50, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/50]	


In [71]:
#test clean accuray
test(val_loader, net_ori)


Accuracy of the network on the 10000 test images: 60 %


In [72]:
#net = BaseNet().to(device)
#test attacks 
pgd_attack = generate_adv(net_ori, "pgd")
fgsm_attack = generate_adv(net_ori, "fgsm")
nifgsm_attack = generate_adv(net_ori, "nifgsm")
vmifgsm_attack = generate_adv(net_ori, "vmifgsm")




In [73]:
images, labels = load_dataset(10000, val_loader)

In [74]:
adv_images_pgd_adv = pgd_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 2.21 %
adv model


In [75]:
adv_images_fgsm_adv = fgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 5.24 %
adv model


In [76]:
#nifgsm_attack = generate_adv(net, "nifgsm")

adv_images_nifgsm_adv = nifgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 3.07 %
adv model


In [77]:
#vmifgsm_attack = generate_adv(net, "vmifgsm")
adv_images_vmifgsm_adv =vmifgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 2.33 %
adv model


# Adversarial Training

In [78]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(net_ori) #am I doing correct? 
for param in net_adv.parameters():
    print(param)

Parameter containing:
tensor([[[[-1.1766e-01, -1.8183e-01, -3.2964e-01, -2.2652e-01, -6.8259e-02],
          [-1.5082e-01, -8.9725e-02, -2.0358e-01, -1.6549e-01,  1.1314e-02],
          [ 8.6422e-02, -5.4958e-02, -3.4714e-01,  1.8159e-01,  1.7917e-01],
          [ 1.2738e-01,  1.2229e-02,  7.0509e-02,  1.5102e-01, -2.7590e-02],
          [-1.1907e-01, -8.6042e-02, -1.0108e-01, -6.6834e-02, -3.8344e-01]],

         [[ 2.2551e-01,  2.0506e-01,  7.2711e-02,  3.5453e-02, -8.3972e-03],
          [ 1.8906e-01,  7.4282e-02,  6.3909e-02,  5.2882e-03,  3.2864e-01],
          [ 1.5228e-01,  2.3276e-01, -1.0917e-01,  3.1298e-01,  4.4610e-01],
          [ 3.7908e-01,  3.2184e-01,  1.8593e-01,  4.2562e-01,  1.6257e-01],
          [ 4.6530e-02,  1.1021e-01,  1.1405e-01,  3.3282e-02,  1.5461e-02]],

         [[-1.2004e-03,  6.8012e-02, -2.2809e-01,  1.3642e-02, -2.1930e-02],
          [-1.8126e-01, -1.4326e-02, -2.4008e-02, -1.5793e-01,  6.4459e-02],
          [ 1.0466e-01, -5.5423e-03, -2.0247e-01, 

In [79]:

    
    
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

In [80]:
train(net_adv, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 50, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/50]	
Epoch [1/50]	 Training Loss: 2.377280056019268	 lr: 0.001
Epoch [1/50]	 Validation Loss: 1.720308845556235	 lr: 0.001
Epoch [1/50]	
Epoch [2/50]	 Training Loss: 2.13429512636131	 lr: 0.001
Epoch [2/50]	 Validation Loss: 1.664982462231117	 lr: 0.001
Epoch [2/50]	
Epoch [3/50]	 Training Loss: 2.085019654020324	 lr: 0.001
Epoch [3/50]	 Validation Loss: 1.649429227732405	 lr: 0.001
Epoch [3/50]	
Epoch [4/50]	 Training Loss: 2.058767602266863	 lr: 0.001
Epoch [4/50]	 Validation Loss: 1.6319529119926164	 lr: 0.001
Epoch [4/50]	
Epoch [5/50]	 Training Loss: 2.040107716684756	 lr: 0.001
Epoch [5/50]	 Validation Loss: 1.62628058693077	 lr: 0.001
Epoch [5/50]	
Epoch [6/50]	 Training Loss: 2.0261903385372113	 lr: 0.001
Epoch [6/50]	 Validation Loss: 1.6111643555798107	 lr: 0.001
Epoch [6/50]	
Epoch [7/50]	 Training Loss: 2.0158568035306224	 lr: 0.001
Epoch [7/50]	 Validation Loss: 1.6108988961086999	 lr: 0.001
Epoch [7/50]	
Epoch [8/50]	 Training Loss: 2.0086441445533576	 lr: 0.001


PGD(model_name=BaseNet, device=cuda:0, return_type=float, attack_mode=default, targeted=False, normalization_used=False, eps=0.03137254901960784, alpha=0.008888888888888889, steps=10, random_start=True)

In [81]:
#test clean accuray
test(val_loader, net_adv)

Accuracy of the network on the 10000 test images: 49 %


In [82]:
acc = clean_accuracy(net_adv, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 40.75 %
adv model


In [83]:
acc = clean_accuracy(net_adv, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 39.76 %
adv model


In [84]:
acc = clean_accuracy(net_adv, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 40.58 %
adv model


In [85]:
acc = clean_accuracy(net_adv, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 40.39 %
adv model


# TODO: Adversarial Training by nifgsm

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv_ni = copy.deepcopy(net_ori) #am I doing correct? 
for param in net_adv_ni.parameters():
    print(param)

In [ ]:

    
    
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_nifgsm")
ni_attack = generate_adv(net_adv_ni, "nifgsm")

In [ ]:
train(net_adv, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 50, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

# TODO: Adversarial Training by vmifgsm

# TODO: Adversarial Training by fgsm